# Taller Turning Grill

**Estudiante:** Omar David Toledo Leguizamón

Implementar el algoritmo de Turning Grille utilizando Python, en donde se reciben los siguientes parámetros:

1. Tamaño de la retícula (cuadrada, por ejemplo 2 indica que es de 2x2)
2. Dirección de la rotación (sentido de las manecillas del reloj o sentido contrario, por ejemplo 1 ó 0 respectivamente)
3. Modo (cifra miento o desciframiento, por ejemplo 1 ó 0 respectivamente)
4. Cantidad y distribución de los hoyos (se recomienda ingresar únicamente las celdas de la matriz en la que hay hoyos)
5. Mensaje a (de)cifrar

## Desarrollo

La primera parte, corresponde a recibir la entrada de la grilla y sus dimensiones correspondientes. La información de la grilla se recibe como un arreglo de unos y ceros de tamaño $n^2$ que se transformará en una matriz booleana $n \times n$ que representará la grilla a rotar

In [3]:
import numpy as np

def buildMatrix(grill,n):
    return  np.array(grill, dtype = bool).reshape((n,n))

In [18]:
t = [1,0,0,0]
n = 2
buildMatrix(t,n)

array([[ True, False],
       [False, False]])

In [4]:
t = [1,0,0,0,0,1,0,1,0]
n = 3
buildMatrix(t,n)

array([[ True, False, False],
       [False, False,  True],
       [False,  True, False]])

Posteriormente, diseñamos una rutina que dada una entrada, la separe en matrices de tamaño $n \times n$ que sea utilizado para el proceso de cifrado o descifrado.

In [5]:
import numpy as np

def matrixInput(text,n):
    text = text.replace(' ','')
    size = n*n
    val = len(text)%size
    if val>0:
        text = text + 'X'*(size-val)
    parts = []
    for i in range(0,len(text),size):
        parts.append(np.array(list(text[i:i+size])).reshape((n,n)))
    return parts

In [6]:
text = 'I BELIEVE WHAT SHOULD I DO'
matrixInput(text,3)

[array([['I', 'B', 'E'],
        ['L', 'I', 'E'],
        ['V', 'E', 'W']], dtype='<U1'),
 array([['H', 'A', 'T'],
        ['S', 'H', 'O'],
        ['U', 'L', 'D']], dtype='<U1'),
 array([['I', 'D', 'O'],
        ['X', 'X', 'X'],
        ['X', 'X', 'X']], dtype='<U1')]

Adicionalmente, definimos una rutina que separe une entrada en subcadenas de tamaño $k$

In [7]:
def splitInput(text, size):
    text = text.replace(' ','')
    val = len(text)%size
    if val>0:
        text = text + 'X'*(size-val)
    parts = []
    for i in range(0,len(text),size):
        parts.append(text[i:i+size])
    return parts

In [8]:
text = 'I BELIEVE WHAT SHOULD I DO'
splitInput(text,3)

['IBE', 'LIE', 'VEW', 'HAT', 'SHO', 'ULD', 'IDO']

Finalmente definimos una rutina que dada la dirección (`0` para ir a favor y `1` para ir en contra de las manecillas del reloj) se encarga de recuperar el mensaje con base a una grilla definida. 

In [9]:
def encodeGrill(input,grill,dir = 0):
    k = 1 if dir==0 else 3
    g = grill.copy()
    result = np.full(grill.shape, 'X', dtype='<U1')
    n = np.sum(grill)
    for i in range(4):
        t = input[i*n:(i+1)*n]
        result[g] = list(t)
        g1 = np.rot90(g,k=k)
        g = np.logical_and(g1,np.logical_not((np.logical_and(g,g1))))
    return ''.join(list(result.reshape(len(grill)**2)))

def decodeGrill(input,grill,dir = 0):
    output = ''
    k = 1 if dir==0 else 3
    g = grill.copy()
    for _ in range(4):
        output += ''.join(list(input[g]))
        g1 = np.rot90(g,k=k)
        g = np.logical_and(g1,np.logical_not((np.logical_and(g,g1))))
    return output

def processInput(input,grill,encode = 0,dir=0):
    call = encodeGrill if encode == 1 else decodeGrill
    s = ''
    for d in input:
        s+=call(d,grill,dir)
    return s

Finalmente, unimos estos procesos en funciones de cifrado y descifrado, tomando como punto de partida que la dirección que se defina es la de cifrado.

In [10]:
def cipherInput(text, grill, dir = 0):
    input = splitInput(text,np.sum(grill)*4)
    return processInput(input,grill,encode=1,dir = dir)

def decipherInput(text, grill, dir = 0):
    input = matrixInput(text,len(grill))
    return processInput(input,grill,encode=0,dir = dir)

In [11]:
t = [1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0]
text = 'I BELIEVE THIS IS WORKING WELL AND I LOVE IT'
n = 4
dir = 0
grill = buildMatrix(t,n)
cipherText = cipherInput(text,grill,dir)
decipherText = decipherInput(cipherText,grill,dir)
print(f'Original Text:\t{text}')
print(f'CipherText:\t{cipherText}')
print(f'Recovered Text:\t{decipherText}')

Original Text:	I BELIEVE THIS IS WORKING WELL AND I LOVE IT
CipherText:	ITIIHSIEWBVEEOLSRLGIALNWOKEILVNDEXXXXXXXXIXTXXXX
Recovered Text:	IBELIEVETHISISWORKINGWELLANDILOVEITXXXXXXXXXXXXX


Finalmente, diseñamos una rutina que reciba el contenido por consola

In [19]:
def TurningGrillMain():
    print('Inicio de programa de cifrado-descifrado usando Turning Grill\n')
    mode = int(input('Elija el modo de operación (1 para cifrar, 2 para descifrar, 3 para salir): '))
    if mode==3: return
    n = int(input('Ingresa el tamaño de la grilla (Ej: 2 si es una grilla 2x2): '))
    g = map( lambda x : int(x) ,input(f'Ingresa los {n*n} valores de la grilla separados por espacios (1 si hay un hoyo y 0 si no lo hay): ').split(' '))
    dir = int(input('Ingresa la dirección de la rotación (0 para ir a favor y 1 para ir en contra de las manecillas del reloj): '))
    grill = buildMatrix(list(g),n)
    if mode==1:
        text = str(input('Ingresa el texto a cifrar: '))
        answer = cipherInput(text,grill,dir)
        print(f'Texto cifrado: {answer}')
    elif mode==2:
        text = str(input('Ingresa el texto a descifrar: '))
        answer = decipherInput(text,grill,dir)
        print(f'Texto descifrado: {answer}')
    else:
        return
    print()
    TurningGrillMain()

In [20]:
TurningGrillMain()

Inicio de programa de cifrado-descifrado usando Turning Grill

Texto cifrado: HoalcoomesatsXXX

Inicio de programa de cifrado-descifrado usando Turning Grill



ValueError: invalid literal for int() with base 10: ''